# Caso Práctico - ENGIE

*Jose Enrique Zafra Mena*

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import openpyxl

## Carga y preparación de Datos


In [3]:
facturas_df=pd.read_excel('facturas.xlsx')
mapping_df=pd.read_excel('Mapping.xlsx')

# renombramos los Deals
CTV_df=pd.read_excel('JVLNG_CTV_STOK_MARZO_2024_05_23_14_12.xlsx')
IAC_df=pd.read_excel('MEDG2_IAC_GETRA_MARZO_2024_05_23_14_12.xlsx')
AOC2_df=pd.read_excel('SUGS2_AOC_STOK_MARZO_2024_05_23_14_12.xlsx')
AOC_df=pd.read_excel('SUGST_AOC_STOK_MARZO_2024_05_23_14_12.xlsx')


In [4]:
# verificamos las primeras filas de cada dataframe
print("Facturas AGIkey:")
print(facturas_df.head())

print("\nMapping:")
print(mapping_df.head())

print("\nCTV:")
print(CTV_df.head())

print("\nIAC:")
print(IAC_df.head())

print("\nAOC2:")
print(AOC2_df.head())

print("\nAOC:")
print(AOC_df.head())

Facturas AGIkey:
   NumeroFactura       Emisor       RazonSocialEmisor  DepEmisor     Receptor  \
0     2024264672  ESA86484292         ENAGAS GTS, SAU       60.0  ESB82508441   
1     2024264671  ESA86484292         ENAGAS GTS, SAU       60.0  ESB82508441   
2     2324004231  ESA86484334  ENAGAS TRANSPORTE, SAU        NaN  ESB82508441   
3     2024164592  ESA86484292         ENAGAS GTS, SAU       50.0  ESB82508441   
4     2024164591  ESA86484292         ENAGAS GTS, SAU       50.0  ESB82508441   

  RazonSocialReceptor  DepReceptor FechaFactura    Importe Moneda    Estado  \
0    ENGIE ESPAÑA SLU          NaN       4/3/24  123195.13    EUR  Recibida   
1    ENGIE ESPAÑA SLU          NaN       4/3/24  111963.62    EUR  Recibida   
2    ENGIE ESPAÑA SLU          NaN       6/3/24  503763.47    EUR  Recibida   
3    ENGIE ESPAÑA SLU          NaN       4/3/24    1375.12    EUR  Recibida   
4    ENGIE ESPAÑA SLU          NaN       4/3/24    1787.43    EUR  Recibida   

  FechaEstado FechaRe

In [5]:
# verificamos las columnas de cada dataframe
print("\nColumnas de Facturas:")
print(facturas_df.columns)

print("\nColumnas de Mapping:")
print(mapping_df.columns)

print("\nColumnas de CTV:")
print(CTV_df.columns)
 
print("\nColumnas de IAC:")
print(IAC_df.columns)

print("\nColumnas de AOC2:")
print(AOC2_df.columns)

print("\nColumnas de AOC:")
print(AOC_df.columns)



Columnas de Facturas:
Index(['NumeroFactura', 'Emisor', 'RazonSocialEmisor', 'DepEmisor', 'Receptor',
       'RazonSocialReceptor', 'DepReceptor', 'FechaFactura', 'Importe',
       'Moneda', 'Estado', 'FechaEstado', 'FechaRegistro', 'Destino',
       'Contrato', 'Origen', 'ServicioFacturado', '¿Verificada por BO?',
       'Observaciones'],
      dtype='object')

Columnas de Mapping:
Index(['Origen', 'Servicio facturado', 'Portfolio', 'Commodity', 'DealType'], dtype='object')

Columnas de CTV:
Index(['Unnamed: 0', 'AggregatedKey', 'ApplicationPeriod', 'BookingSource',
       'BOValidated', 'BOValidationDate', 'BOValidator', 'CancelsDealId',
       'Capacity', 'CapacityFrequency',
       ...
       'ExerciseDate', 'ExerciseType', 'Nature', 'OptionNature', 'OptionType',
       'PeriodInfoCollection', 'Script', 'TemplateId', 'TemplateScript',
       'OrderId'],
      dtype='object', length=108)

Columnas de IAC:
Index(['Unnamed: 0', 'AggregatedKey', 'ApplicationPeriod', 'AuctionType',
   

Vamos a renombrar columnas comunes,

In [6]:
facturas_df.rename(columns={'ServicioFacturado': 'Servicio Facturado'}, inplace=True)
mapping_df.rename(columns={'Servicio facturado': 'Servicio Facturado'}, inplace=True)

## Transformación de los datos

Los Deals tienen columnas que no necesitamos para la conciliación. Necesitamos solo algo que lo identifique (ID) y su importe. 

In [19]:
# para renombrar las columnas, usamos las columnas de CTV
CTV_min_df = CTV_df.loc[:,['Id','LegId','LegType','Category', 'TotalQuantity', 'TradeDate']]
# les ponemos otro nombre
CTV_min_df.rename(columns={'TotalQuantity':'Quantity' } , inplace=True)

# ahora escogemos las columnas que queremos de cada Deal
IAC_min_df = IAC_df.loc[:,['Id','LegId','LegType','Category', 'Quantity', 'TradeDate']]
AOC_min_df = AOC_df.loc[:,['Id','LegId','LegType','Category', 'Quantity', 'TradeDate']]
AOC2_min_df = AOC2_df.loc[:,['Id','LegId','LegType','Category', 'Quantity', 'TradeDate']]


Vamos a conectar el Mapping con cada Deal, ya que queremos comparar con las facturas.

Para conectar Mapping con CTV usamos Commodity==CTV

Para conectar Mapping con IAC usamos Commodity==IAC

Para conectar Mapping con AOC usamos Commodity==AOC & Portofolio: SUGST

Para conectar Mapping con AOC2 usamos Commodity==AOC & Portofolio: SUGS2


In [36]:
# para poder unir cada Deal con Mapping individualmente, añadimos las columnas Portofolio y Commodity a cada deal
# y le asignamos su nombre correspondiente

CTV_min_df['Portfolio']='JVLNG'
CTV_min_df['Commodity']='CTV'


IAC_min_df['Portfolio']='MEDG2'
IAC_min_df['Commodity']='IAC'

AOC_min_df['Portfolio']='SUGST'
AOC_min_df['Commodity']='AOC'

AOC2_min_df['Portfolio']='SUGS2'
AOC2_min_df['Commodity']='AOC'

IAC_min_df.head()

,Id,LegId,LegType,Category,Quantity,TradeDate,Portfolio,Commodity
0,37591650,37591650,Transport,O,0,2023-02-15T00:00:00+01:00,MEDG2,IAC
1,37056084,37056084,Transport,O,0,2022-12-29T00:00:00+01:00,MEDG2,IAC


In [78]:
# ahora unimos cada uno con mapping

CTV_mapped_df = pd.merge(CTV_min_df,mapping_df,on=['Commodity', 'Portfolio'],how='inner')
IAC_mapped_df = pd.merge(IAC_min_df,mapping_df,on=['Commodity', 'Portfolio'],how='inner')
AOC_mapped_df = pd.merge(AOC_min_df, mapping_df, on=['Commodity', 'Portfolio'], how='inner')
AOC2_mapped_df = pd.merge(AOC2_min_df,mapping_df,on=['Commodity', 'Portfolio'],how='inner')

#print(CTV_mapped_df.head())
CTV_mapped_df.head()

,Id,LegId,LegType,Category,Quantity,TradeDate,Portfolio,Commodity,Origen,Servicio Facturado,DealType
0,42563586,42563586.0,Delivery,O,90000.0,2024-04-29T00:00:00+02:00,JVLNG,CTV,TVB,Almacenamiento TVB,STOK
1,42563586,42563586.0,Delivery,O,90000.0,2024-04-29T00:00:00+02:00,JVLNG,CTV,TVB,Almacenamiento TVB,STOK
2,42563586,42563586.0,Delivery,O,90000.0,2024-04-29T00:00:00+02:00,JVLNG,CTV,TVB,Almacenamiento TVB,STOK
3,42518127,42518127.0,Delivery,O,50000.0,2024-04-25T00:00:00+02:00,JVLNG,CTV,TVB,Almacenamiento TVB,STOK
4,42518127,42518127.0,Delivery,O,50000.0,2024-04-25T00:00:00+02:00,JVLNG,CTV,TVB,Almacenamiento TVB,STOK


In [48]:
# Por último unimos todos los deals en un solo datafram
deals_mapped_df = pd.concat([CTV_mapped_df, IAC_mapped_df, AOC_mapped_df, AOC2_mapped_df], ignore_index=True)

deals_mapped_df.head(15)

,Id,LegId,LegType,Category,Quantity,TradeDate,Portfolio,Commodity,Origen,Servicio Facturado,DealType
0,42563586,42563586.0,Delivery,O,90000.0,2024-04-29T00:00:00+02:00,JVLNG,CTV,TVB,Almacenamiento TVB,STOK
1,42563586,42563586.0,Delivery,O,90000.0,2024-04-29T00:00:00+02:00,JVLNG,CTV,TVB,Almacenamiento TVB,STOK
2,42563586,42563586.0,Delivery,O,90000.0,2024-04-29T00:00:00+02:00,JVLNG,CTV,TVB,Almacenamiento TVB,STOK
3,42518127,42518127.0,Delivery,O,50000.0,2024-04-25T00:00:00+02:00,JVLNG,CTV,TVB,Almacenamiento TVB,STOK
4,42518127,42518127.0,Delivery,O,50000.0,2024-04-25T00:00:00+02:00,JVLNG,CTV,TVB,Almacenamiento TVB,STOK
5,42518127,42518127.0,Delivery,O,50000.0,2024-04-25T00:00:00+02:00,JVLNG,CTV,TVB,Almacenamiento TVB,STOK
6,42518126,42518126.0,Delivery,O,50000.0,2024-04-25T00:00:00+02:00,JVLNG,CTV,TVB,Almacenamiento TVB,STOK
7,42518126,42518126.0,Delivery,O,50000.0,2024-04-25T00:00:00+02:00,JVLNG,CTV,TVB,Almacenamiento TVB,STOK
8,42518126,42518126.0,Delivery,O,50000.0,2024-04-25T00:00:00+02:00,JVLNG,CTV,TVB,Almacenamiento TVB,STOK
9,42518125,42518125.0,Delivery,O,50000.0,2024-04-25T00:00:00+02:00,JVLNG,CTV,TVB,Almacenamiento TVB,STOK


Con esto ya tenemos los Deals listos para compararlos con Facturas (Conciliación)

## Conciliación

In [35]:
# quitamos columnas de Facturas que no son relevantes para este análisis
facturas_min_df = facturas_df.loc[:, ['NumeroFactura', 'Importe', 'FechaFactura', 'Origen', 'Servicio Facturado']]

#añadimos una columna para decir si los valores están conciliados o no, y otra de observaciones para más adelante
facturas_min_df['Conciliado'] = False
facturas_min_df['Observaciones'] = ""

facturas_min_df.head()

,NumeroFactura,Importe,FechaFactura,Origen,Servicio Facturado,Conciliado,Observaciones
0,2024264672,123195.13,4/3/24,AVB,Servicio agregado AVB,False,
1,2024264671,111963.62,4/3/24,AVB,Servicio agregado AVB,False,
2,2324004231,503763.47,6/3/24,C.I. Almería,Entrada PVB,False,
3,2024164592,1375.12,4/3/24,TVB,Almacenamiento TVB,False,
4,2024164591,1787.43,4/3/24,TVB,Almacenamiento TVB,False,


In [51]:
# Ahora hacemos un merge con las columnas Origen y Servicio Facturado
facturas_conciliadas_df = facturas_min_df.merge(deals_mapped_df, on=['Origen', 'Servicio Facturado'], how='outer')

print(facturas_conciliadas_df.shape)
facturas_conciliadas_df.head(1522)

(1521, 16)


,NumeroFactura,Importe,FechaFactura,Origen,Servicio Facturado,Conciliado,Observaciones,Id,LegId,LegType,Category,Quantity,TradeDate,Portfolio,Commodity,DealType
0,NaN,NaN,NaN,AVB,Almacenamiento AVB,NaN,NaN,41888250,41888250.0,Storage,O,800000.000,2024-02-22T00:00:00+01:00,SUGST,AOC,STOK
1,NaN,NaN,NaN,AVB,Almacenamiento AVB,NaN,NaN,38290460,38290460.0,Delivery,O,924.588,2023-04-20T00:00:00+02:00,SUGST,AOC,STOK
2,NaN,NaN,NaN,AVB,Almacenamiento AVB,NaN,NaN,38249489,38249489.0,Delivery,O,352.340,2023-04-17T00:00:00+02:00,SUGST,AOC,STOK
3,NaN,NaN,NaN,AVB,Almacenamiento AVB,NaN,NaN,38054569,38054569.0,Delivery,O,3910.694,2023-03-24T00:00:00+01:00,SUGST,AOC,STOK
4,NaN,NaN,NaN,AVB,Almacenamiento AVB,NaN,NaN,36525979,36525979.0,Storage,O,800000.000,2022-11-16T00:00:00+01:00,SUGST,AOC,STOK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1516,2.024165e+09,59080.41,4/3/24,TVB,Almacenamiento TVB,False,,34833331,34833331.0,Storage,O,NaN,2022-05-24T00:00:00+02:00,JVLNG,CTV,STOK
1517,2.024165e+09,59080.41,4/3/24,TVB,Almacenamiento TVB,False,,34833331,34833331.0,Storage,O,NaN,2022-05-24T00:00:00+02:00,JVLNG,CTV,STOK
1518,2.024165e+09,59080.41,4/3/24,TVB,Almacenamiento TVB,False,,31915597,31915597.0,Storage,O,NaN,2021-08-12T00:00:00+02:00,JVLNG,CTV,STOK
1519,2.024165e+09,59080.41,4/3/24,TVB,Almacenamiento TVB,False,,31915597,31915597.0,Storage,O,NaN,2021-08-12T00:00:00+02:00,JVLNG,CTV,STOK


In [77]:
# Ahora filtramos por la fecha. Primero los ponemos todas en el mismo formato

facturas_conciliadas_df['FechaFactura'] = pd.to_datetime(facturas_conciliadas_df['FechaFactura'], dayfirst=True, format="%d/%m/%Y")
facturas_conciliadas_df['TradeDate'] = pd.to_datetime(facturas_conciliadas_df['TradeDate'], utc=True, dayfirst=True)

# Filtramos para tener solo los trades de Marzo 2024 (ESTO PUEDE SER UN ERROR)
facturas_conciliadas_Marzo_df = facturas_conciliadas_df[(facturas_conciliadas_df['TradeDate'].dt.year == 2024) & (facturas_conciliadas_df['TradeDate'].dt.month == 3)]

facturas_conciliadas_Marzo_df.head(1000)

,NumeroFactura,Importe,FechaFactura,Origen,Servicio Facturado,Conciliado,Observaciones,Id,LegId,LegType,Category,Quantity,TradeDate,Portfolio,Commodity,DealType
192,2.024165e+09,1375.12,2024-04-03,TVB,Almacenamiento TVB,False,,42238975,42238975.0,Delivery,O,15000.0,2024-03-25 00:00:00+00:00,JVLNG,CTV,STOK
193,2.024165e+09,1375.12,2024-04-03,TVB,Almacenamiento TVB,False,,42238975,42238975.0,Delivery,O,15000.0,2024-03-25 00:00:00+00:00,JVLNG,CTV,STOK
194,2.024165e+09,1375.12,2024-04-03,TVB,Almacenamiento TVB,False,,42238975,42238975.0,Delivery,O,15000.0,2024-03-25 00:00:00+00:00,JVLNG,CTV,STOK
195,2.024165e+09,1375.12,2024-04-03,TVB,Almacenamiento TVB,False,,42238679,42238679.0,Delivery,O,45000.0,2024-03-25 00:00:00+00:00,JVLNG,CTV,STOK
196,2.024165e+09,1375.12,2024-04-03,TVB,Almacenamiento TVB,False,,42238679,42238679.0,Delivery,O,45000.0,2024-03-25 00:00:00+00:00,JVLNG,CTV,STOK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1462,2.024165e+09,59080.41,2024-04-03,TVB,Almacenamiento TVB,False,,42238677,42238677.0,Delivery,O,31000.0,2024-03-25 00:00:00+00:00,JVLNG,CTV,STOK
1463,2.024165e+09,59080.41,2024-04-03,TVB,Almacenamiento TVB,False,,42238677,42238677.0,Delivery,O,31000.0,2024-03-25 00:00:00+00:00,JVLNG,CTV,STOK
1464,2.024165e+09,59080.41,2024-04-03,TVB,Almacenamiento TVB,False,,42238676,42238676.0,Delivery,O,91000.0,2024-03-25 00:00:00+00:00,JVLNG,CTV,STOK
1465,2.024165e+09,59080.41,2024-04-03,TVB,Almacenamiento TVB,False,,42238676,42238676.0,Delivery,O,91000.0,2024-03-25 00:00:00+00:00,JVLNG,CTV,STOK


## Propuesta de Valor